In [3]:
from tensorflow import keras

In [4]:
import os
import numpy as np
import cv2
import nibabel as nib
#%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from skimage.transform import resize
from mpl_toolkits.mplot3d import Axes3D

import scipy.ndimage
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate, Add, Dropout)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import pandas as pd

In [5]:
# Function to load NIfTI files with memory mapping
def load_nifti_memmap(file_path):
    img = nib.load(file_path)
    data = img.get_fdata(dtype=np.float32, caching='unchanged')  # Memory-mapped array
    affine = img.affine
    header = img.header
    return data, affine, header

# Generator function to load data in batches
def data_generator(file_list, data_path, mask_path, batch_size, target_shape=None):
    while True:
        np.random.shuffle(file_list)
        for start in range(0, len(file_list), batch_size):
            end = min(start + batch_size, len(file_list))
            batch_files = file_list[start:end]
            
            X_batch = []
            y_batch = []
            
            for filename in batch_files:
                img_path = os.path.join(data_path, filename)
                corresponding_mask_path = os.path.join(mask_path, filename)
                
                image, _, _ = load_nifti_memmap(img_path)
                mask, _, _ = load_nifti_memmap(corresponding_mask_path)
                
                # Ensure image and mask have the same shape (and possibly resize if needed)
                if target_shape:
                    image = resize_volume(image, target_shape)
                    mask = resize_volume(mask, target_shape)
                
                X_batch.append(image)
                y_batch.append(mask)
            
            X_batch = np.array(X_batch)[..., np.newaxis]  # Adding channel dimension
            y_batch = np.array(y_batch)[..., np.newaxis]  # Adding channel dimension
            
            yield X_batch, y_batch

# Function to resize volumes (if needed)
def resize_volume(img, target_shape):
    current_shape = img.shape
    if current_shape == target_shape:
        return img
    # Example: using scipy for interpolation
    resized_img = scipy.ndimage.zoom(img, (target_shape[0]/current_shape[0], target_shape[1]/current_shape[1], target_shape[2]/current_shape[2]), order=3)
    return resized_img

def resize_image(image, target_shape):
    # Resize the image to match the target shape (height, width)
    return resize(image, target_shape, preserve_range=True, anti_aliasing=True)

def pad_or_crop_volume(volume, target_shape):
    current_shape = volume.shape
    
    # Calculate padding width for each dimension
    pad_width = [(0, max(target_shape[i] - current_shape[i], 0)) for i in range(3)]
    
    # Pad the volume to the target shape
    volume = np.pad(volume, pad_width, mode='constant', constant_values=0)
    
    # Calculate cropping dimensions for each dimension
    crop_start = [(volume.shape[i] - target_shape[i]) // 2 for i in range(3)]
    crop_end = [crop_start[i] + target_shape[i] for i in range(3)]
    
    # Crop the volume to the target shape
    slices = [slice(crop_start[i], crop_end[i]) for i in range(3)]
    volume = volume[slices[0], slices[1], slices[2]]
    
    return volume

def calculate_volume(mask, voxel_volume):
    # Calculate the volume based on the mask
    return np.sum(mask) * voxel_volume

In [6]:
# Metrics
def dice_similarity(y_true, y_pred):
    smooth = 1.0
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def jaccard_index(y_true, y_pred):
    smooth = 1.0
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

def boundary_f1_score(y_true, y_pred, distance_threshold=1.5):
    """
    Approximate Boundary F1 Score: Measures how well predicted and true boundaries match.
    For simplicity, this assumes binary masks and uses Manhattan distance to calculate boundary proximity.
    """
    def boundaries(mask):
        """Extract boundaries from a binary mask."""
        edge_x = K.abs(mask[:, 1:, :, :] - mask[:, :-1, :, :])
        edge_y = K.abs(mask[:, :, 1:, :] - mask[:, :, :-1, :])
        edge_z = K.abs(mask[:, :, :, 1:] - mask[:, :, :, :-1])
        return K.clip(edge_x + edge_y + edge_z, 0, 1)
    
    true_boundary = boundaries(y_true)
    pred_boundary = boundaries(y_pred)
    
    true_positive = K.sum(true_boundary * pred_boundary)
    false_positive = K.sum(pred_boundary * (1 - true_boundary))
    false_negative = K.sum(true_boundary * (1 - pred_boundary))
    
    precision = true_positive / (true_positive + false_positive + K.epsilon())
    recall = true_positive / (true_positive + false_negative + K.epsilon())
    
    return 2 * (precision * recall) / (precision + recall + K.epsilon())

# Loss Function
def dice_loss(y_true, y_pred):
    return 1 - dice_similarity(y_true, y_pred)

def combined_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return bce + dice

In [7]:



# Attention Block
def attention_block(x, g, inter_channel):
    theta_x = Conv3D(inter_channel, (1, 1, 1))(x)
    phi_g = Conv3D(inter_channel, (1, 1, 1))(g)
    add = Add()([theta_x, phi_g])
    psi = tf.keras.activations.relu(add)
    psi = Conv3D(1, (1, 1, 1), activation='sigmoid')(psi)
    return tf.multiply(x, psi)

# Residual Block
def residual_block(input_tensor, filters):
    conv = Conv3D(filters, (3, 3, 3), activation='relu', padding='same')(input_tensor)
    conv = Conv3D(filters, (3, 3, 3), activation='relu', padding='same')(conv)
    shortcut = Conv3D(filters, (1, 1, 1), padding='same')(input_tensor)
    output = Add()([conv, shortcut])
    return output

# Modified 3D U-Net
def unet_3d_modified(input_shape):
    inputs = Input(input_shape)
    
    # Downsampling
    c1 = residual_block(inputs, 32)
    p1 = MaxPooling3D((2, 2, 2))(c1)
    
    c2 = residual_block(p1, 64)
    p2 = MaxPooling3D((2, 2, 2))(c2)
    
    c3 = residual_block(p2, 128)
    p3 = MaxPooling3D((2, 2, 2))(c3)
    
    c4 = residual_block(p3, 256)  # Bottleneck
    c4 = Dropout(0.3)(c4)  # Dropout for regularization
    
    # Upsampling
    u5 = UpSampling3D((2, 2, 2))(c4)
    u5 = attention_block(u5, c3, 128)  # Attention
    u5 = concatenate([u5, c3])
    c5 = residual_block(u5, 128)
    
    u6 = UpSampling3D((2, 2, 2))(c5)
    u6 = attention_block(u6, c2, 64)  # Attention
    u6 = concatenate([u6, c2])
    c6 = residual_block(u6, 64)
    
    u7 = UpSampling3D((2, 2, 2))(c6)
    u7 = attention_block(u7, c1, 32)  # Attention
    u7 = concatenate([u7, c1])
    c7 = residual_block(u7, 32)
    
    outputs = Conv3D(1, (1, 1, 1), activation='sigmoid')(c7)
    
    return Model(inputs, outputs)

# Dice + BCE Loss Function
def dice_loss(y_true, y_pred):
    smooth = 1.0
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def combined_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return bce + dice

# Compile the Model
input_shape = (128, 128, 200, 1)  # Example input shape
model = unet_3d_modified(input_shape)
model.compile(optimizer=Adam(learning_rate=1e-4), loss=combined_loss, metrics=['accuracy', dice_similarity, jaccard_index, boundary_f1_score])

model.summary()


2025-01-21 09:55:53.126044: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43391 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:51:00.0, compute capability: 8.6


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 200, 1)]   0         []                            
                                                                                                  
 conv3d (Conv3D)             (None, 128, 128, 200, 32)    896       ['input_1[0][0]']             
                                                                                                  
 conv3d_1 (Conv3D)           (None, 128, 128, 200, 32)    27680     ['conv3d[0][0]']              
                                                                                                  
 conv3d_2 (Conv3D)           (None, 128, 128, 200, 32)    64        ['input_1[0][0]']             
                                                                                              

In [8]:
# Paths for data and masks
train_data_path = r'/home/icmr/Documents/JIP_DATASET/PORTAL/Train/imageTr'
train_mask_path = r'/home/icmr/Documents/JIP_DATASET/PORTAL/Train/LabelTr'

val_data_path = r'/home/icmr/Documents/JIP_DATASET/PORTAL/Val/imageVal'
val_mask_path = r'/home/icmr/Documents/JIP_DATASET/PORTAL/Val/LabelVal'

test_data_path = r'/home/icmr/Documents/JIP_DATASET/PORTAL/Test/imageTs'
test_mask_path = r'/home/icmr/Documents/JIP_DATASET/PORTAL/Test/LabelTs'

In [9]:
# Verify paths exist
for path in [train_data_path, train_mask_path, val_data_path, val_mask_path, test_data_path, test_mask_path]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Path not found: {path}")

# File lists for each dataset
train_files = [filename for filename in os.listdir(train_data_path) if filename.endswith('.nii.gz')]
val_files = [filename for filename in os.listdir(val_data_path) if filename.endswith('.nii.gz')]
test_files = [filename for filename in os.listdir(test_data_path) if filename.endswith('.nii.gz')]

batch_size = 1

# Data generators
train_generator = data_generator(train_files, train_data_path, train_mask_path, batch_size, target_shape=input_shape[:3])
val_generator = data_generator(val_files, val_data_path, val_mask_path, batch_size, target_shape=input_shape[:3])
test_generator = data_generator(test_files, test_data_path, test_mask_path, batch_size, target_shape=input_shape[:3])


# Calculate steps per epoch
steps_per_epoch = len(train_files) // batch_size
validation_steps = len(val_files) // batch_size
test_steps = len(test_files) // batch_size

# Save the model
model.save('unet3d_new_model.h5')

/home/icmr/myenv_py311/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


: 

In [ ]:
# Define custom callback for metrics
class MetricsLogger(Callback):
    def __init__(self, val_generator, val_steps, test_generator, test_steps):
        super().__init__()
        self.val_generator = val_generator
        self.val_steps = val_steps
        self.test_generator = test_generator
        self.test_steps = test_steps
        self.logs = []

    def on_epoch_end(self, epoch, logs=None):
        # Evaluate on validation set
        val_loss, val_accuracy, val_dice, val_jaccard, val_boundary= self.model.evaluate(
            self.val_generator, steps=self.val_steps, verbose=0
        )
        # Evaluate on test set
        test_loss, test_accuracy, test_dice, test_jaccard, test_boundary = self.model.evaluate(
            self.test_generator, steps=self.test_steps, verbose=0
        )
        # Log metrics
        self.logs.append({
            'epoch': epoch + 1,
            'train_loss': logs.get('loss'),
            'train_accuracy': logs.get('accuracy'),
            'val_loss': val_loss,
            'val_accuracy': val_accuracy,
            'val_dice': val_dice,
            'val_jaccard': val_jaccard,
            'val_boundary' : val_boundary,
            'test_loss': test_loss,
            'test_accuracy': test_accuracy,
            'test_dice': test_dice,
            'test_jaccard': test_jaccard,
            'test_boundary' : test_boundary
        })
        print(f"Epoch {epoch + 1}: "
              f"Train Loss: {logs.get('loss')}, Train Accuracy: {logs.get('accuracy')}, "
              f"Val Loss: {val_loss}, Val Accuracy: {val_accuracy}, "
              f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Define callbacks
checkpoint = ModelCheckpoint('unet3d_new_model.h5', save_best_only=True, monitor='val_loss', mode='min')
metrics_logger = MetricsLogger(val_generator, validation_steps, test_generator, test_steps)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[checkpoint, metrics_logger, early_stopping]
)

# Final evaluation on test set
final_test_loss, final_test_accuracy, final_test_dice, final_test_jaccard, final_boundary_f1_score = model.evaluate(
    test_generator, steps=test_steps
)
print("\nFinal Test Metrics:")
print(f"Test Loss: {final_test_loss}")
print(f"Test Accuracy: {final_test_accuracy}")
print(f"Test Dice Coefficient: {final_test_dice}")
print(f"Test Jaccard Index: {final_test_jaccard}")
print(f"Test Boundary F1 Score: {final_boundary_f1_score}")

# Save metrics logs

metrics_df = pd.DataFrame(metrics_logger.logs)
metrics_df.to_csv('metrics_log.csv', index=False)

# Display metrics as a table
print("\nMetrics per Epoch:")
print(metrics_df)


Epoch 1/50
